In [1]:
import osmnx as ox
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
from pyproj import CRS

In [2]:
def tor_green_index(place='City of Toronto, Golden Horseshoe, Ontario, Canada',
                    shapefile='Neighbourhoods/Neighbourhoods.shp', tag_type='landuse', tag_feat='grass'):
    
    # Reading the neighbourhood shapefile
    neigh = gpd.read_file(shapefile)

    # Location Selection
    tags = {tag_type: tag_feat}
    grass = ox.geometries_from_place(place, tags)

    # Landuses to shapefile
    neigh_grass = gpd.sjoin(neigh, grass, how='left', op='intersects')

    # Grouping spatially joined grass landuses and counting them per neighbourhood
    grouped_grass = neigh_grass.groupby('FIELD_7').count()
    grouped_grass.reset_index(level=0, inplace=True)

    # Eliminating unnecessary columns
    grouped_grass = grouped_grass[['FIELD_7', 'landuse']]

    # Reintroducing geometries in neigh_merge and merging with grouped_grass
    neigh_merge = neigh[['FIELD_7', 'geometry']]
    merged_df = neigh_merge.merge(grouped_grass, left_on='FIELD_7', right_on='FIELD_7')

    # Calculating Density
    merged_df['area'] = merged_df['geometry'].area
    merged_df['density'] = merged_df['landuse']/merged_df['area']
    merged_df['density'] = merged_df['density'].apply(lambda x: x//1000)

    # Plotting
    fig, ax = plt.subplots(figsize=(12,8))
    merged_df.plot(ax=ax,
           column='density',
           linewidth=0.03,
           cmap='Greens',
           scheme='quantiles',
           legend=True,
           k=4,
           alpha=0.9
                  )

    # Re-position the legend and set a title
    ax.get_legend().set_bbox_to_anchor((1.3,1))
    ax.get_legend().set_title('Density per Sq.Km.')
    plt.title('Density of Green Spaces in the City of Toronto')

    plt.tight_layout()
    
    return 